In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy
import matplotlib.pyplot as plt
import cvxpy as cp
import vectorized_sdr as vect_sdr

np.random.seed(0)

In [23]:
N_omega = 11
N_z = 2
n = 10**-5
omega = np.linspace(-3, 3, N_omega)
z = np.linspace(0, 10**-3, N_z)
delta_omega = np.abs(omega[1] - omega[0])
delta_z = np.abs(z[1] - z[0])
green_f = [np.diag(np.exp(1.j*omega*z[i])) for i in range(N_z)]
green_f.reverse()
green_f[0] = 0.5*green_f[0]
green_f[-1] = 0.5*green_f[-1]
beta_up = delta_omega*np.random.random(N_omega)
beta_down = delta_omega*np.random.random(N_omega)
vectorized_beta = np.concatenate((beta_up, beta_down[1:]))
beta = scipy.linalg.hankel(beta_up, beta_down)
delta_k = np.diag(1.j*omega)
Q_plus = delta_k + beta
Q_minus = delta_k - beta
W_plus = [scipy.linalg.expm(Q_plus*z[i]) for i in range(N_z)]
W_minus = [scipy.linalg.expm(Q_minus*z[i]) for i in range(N_z)]
# Try only defining the last J
J = 0.25*(W_plus[-1]@W_plus[-1].conj().T + W_minus[-1]@W_minus[-1].conj().T - 2*np.eye(N_omega))
vectorized_W_plus = sparse.vstack([vect_sdr.mat_to_vec(W_plus[i]) for i in range(N_z)])
vectorized_W_minus = sparse.vstack([vect_sdr.mat_to_vec(W_minus[i]) for i in range(N_z)])
vectorized_J = vect_sdr.mat_to_vec(J)
x = sparse.vstack([vectorized_J, vectorized_W_plus, vectorized_W_minus, sparse.csc_matrix(vectorized_beta).T])
y = sparse.vstack([x, 1])

In [24]:
semi_def_constr = sparse.eye(1)
semi_def_constr = sparse.bmat([[sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,1))],[sparse.csc_matrix((1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), semi_def_constr]])


In [25]:
projection = np.zeros((N_omega, N_omega))
J_def_constr = []
real_plus_dyn = []
imag_plus_dyn = []
real_minus_dyn = []
imag_minus_dyn = []
plus_verif_list = []
minus_verif_list = []
for i in range(N_omega):
    for j in range(N_omega):
        proj = projection.copy()
        proj[i, j] = 1
        real_plus_mats, imag_plus_mats, real_minus_mats, imag_minus_mats = vect_sdr.dynamics_mat(omega, z, proj)
        plus_verif, minus_verif = vect_sdr.verif_constr(W_plus, W_minus, beta, proj, omega, z)
        J_def_constr.append(vect_sdr.J_def_sdp_mat(N_omega, N_z, proj))
        plus_verif_list += plus_verif
        minus_verif_list += minus_verif
        real_plus_dyn += real_plus_mats
        imag_plus_dyn += imag_plus_mats
        real_minus_dyn += real_minus_mats
        imag_minus_dyn += imag_minus_mats
photon_numb_constr = vect_sdr.photon_numb_mat(N_omega, N_z, n)
constr_mats = J_def_constr + real_plus_dyn + imag_plus_dyn + real_minus_dyn + imag_minus_dyn
constr_mats.append(photon_numb_constr)
constr_mats.append(semi_def_constr)
test_W_plus = [(y.conj().T@real_plus_dyn[i]@y).toarray()[0,0] + 1.j*(y.conj().T@imag_plus_dyn[i]@y).toarray()[0,0] for i in range(len(real_plus_dyn))]
test_W_minus = [(y.conj().T@real_minus_dyn[i]@y).toarray()[0,0] + 1.j*(y.conj().T@imag_minus_dyn[i]@y).toarray()[0,0] for i in range(len(real_plus_dyn))]

In [30]:
[(y.conj().T@constr_mats[i]@y).toarray()[0,0] for i in range(len(constr_mats)) if np.abs((y.conj().T@constr_mats[i]@y).toarray()[0,0]) > 10**-5]

[(1+0j)]

In [31]:
vec_to_hankel = vect_sdr.omega_vec_to_omega_dof(N_omega)
lin = vect_sdr.vectorized_proj(N_omega, N_z, proj)
dynamics_real_list = [sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1, (2*N_z + 1)*(N_omega**2) + 2*N_omega - 1))]
dynamics_imag_list = [sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1, (2*N_z + 1)*(N_omega**2) + 2*N_omega - 1))]
for i in range(1, N_z):
    green_fs = vect_sdr.get_green_f(omega, z[:i + 1])
    projected_green_fs = [sparse.csc_matrix(green_fs[i]@proj) for i in range(len(green_fs))]
    full_proj_green_fs = [delta_z*sparse.kron(sparse.eye(N_omega), projected_green_fs[i]) for i in range(len(green_fs))]
    rect_proj_green_fs = [vec_to_hankel.conj().T@full_proj_green_fs[i] for i in range(len(green_fs))]
    rect_proj_green_fs.reverse()
    rect_proj_green_fs[0] = 0.5*rect_proj_green_fs[0]
    rect_proj_green_fs[-1] = 0.5*rect_proj_green_fs[-1]
    # Good until here
    stacked_dynamics = sparse.hstack(rect_proj_green_fs)
    stacked_dynamics.resize((2*N_omega - 1, (2*N_z*(N_omega**2) + 2*N_omega - 1)))
    stacked_dynamics = sparse.hstack([sparse.csc_matrix((2*N_omega - 1, N_omega**2)), stacked_dynamics])
    stacked_dynamics = sparse.vstack([sparse.csc_matrix(((2*N_z + 1)*(N_omega**2), (2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), stacked_dynamics])
    dynamics_real_list.append(0.5*(stacked_dynamics + stacked_dynamics.conj().T))
    dynamics_imag_list.append(-0.5*1.j*(stacked_dynamics - stacked_dynamics.conj().T))

In [32]:
vec_to_hankel = vect_sdr.omega_vec_to_omega_dof(N_omega)
plus_dyn_real, plus_dyn_imag = vect_sdr.dynamics_W(omega, z, proj, vec_to_hankel, "plus")
minus_dyn_real, minus_dyn_imag = vect_sdr.dynamics_W(omega, z, proj, vec_to_hankel, "minus")
full_proj_plus = vect_sdr.vectorized_proj(N_omega, N_z, proj)[1:N_z + 1]
full_proj_minus = vect_sdr.vectorized_proj(N_omega, N_z, proj)[N_z + 1:2*N_z + 1]
green_f = vect_sdr.get_green_f(omega, z)
real_plus_mats_test = [sparse.bmat([[plus_dyn_real[i], -0.5*full_proj_plus[i]],[-0.5*full_proj_plus[i].conj().T, np.real(np.trace(proj@green_f[i]))]]) for i in range(N_z)]
imag_plus_mats_test = [sparse.bmat([[plus_dyn_imag[i], -0.5*1.j*full_proj_plus[i]],[0.5*1.j*full_proj_plus[i].conj().T, np.imag(np.trace(proj@green_f[i]))]]) for i in range(N_z)]
real_minus_mats_test = [sparse.bmat([[minus_dyn_real[i], -0.5*full_proj_minus[i]],[-0.5*full_proj_minus[i].conj().T, np.real(np.trace(proj@green_f[i]))]]) for i in range(N_z)]
imag_minus_mats_test = [sparse.bmat([[minus_dyn_imag[i], -0.5*1.j*full_proj_minus[i]],[0.5*1.j*full_proj_minus[i].conj().T, np.imag(np.trace(proj@green_f[i]))]]) for i in range(N_z)]

In [34]:
# Try adding a covariance penalty
X = cp.Variable(shape=((2*N_z + 1)*(N_omega**2) + 2*N_omega,(2*N_z + 1)*(N_omega**2) + 2*N_omega), hermitian = True)
obj_f_mat = vect_sdr.obj_f_sdp_mat(N_omega, N_z, n)
constraints = [X >> 0]
constraints += [cp.real(cp.trace(J_def_constr[i]@X)) == 0 for i in range(len(J_def_constr))]
constraints += [cp.real(cp.trace(real_plus_dyn[i]@X)) == 0 for i in range(len(real_plus_dyn))]
constraints += [cp.real(cp.trace(imag_plus_dyn[i]@X)) == 0 for i in range(len(real_plus_dyn))]
constraints += [cp.real(cp.trace(real_minus_dyn[i]@X)) == 0 for i in range(len(real_plus_dyn))]
constraints += [cp.real(cp.trace(imag_minus_dyn[i]@X)) == 0 for i in range(len(real_plus_dyn))]
constraints.append(cp.real(cp.trace(photon_numb_constr@X)) == 0)
constraints.append(cp.real((cp.trace(semi_def_constr@X))) == 1)
problem = cp.Problem(cp.Minimize(cp.real(cp.trace(obj_f_mat@X))), constraints)

In [35]:
problem.solve(verbose = True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 06 05:05:56 PM: Your problem has 393129 variables, 1092 constraints, and 0 parameters.
(CVXPY) Feb 06 05:05:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 06 05:05:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 06 05:05:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 06 05:05:56 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 06 05:05:56 PM: Compiling problem (target solver=SCS)

(CVXPY) Feb 06 05:05:56 PM: Applying reduction Dcp2Cone
(CVXPY) Feb 06 05:05:56 PM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 06 05:05:57 PM: Applying reduction Dcp2Cone
(CVXPY) Feb 06 05:05:57 PM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 06 05:05:57 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Feb 06 05:07:02 PM: Applying reduction SCS
(CVXPY) Feb 06 05:07:03 PM: Finished problem compilation (took 6.674e+01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Feb 06 05:07:03 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3

-inf